In [173]:
import yaml
import psycopg2
from yaml.loader import SafeLoader
from sqlalchemy import create_engine   
from database_utils import DatabaseConnector
import pandas as pd
from sqlalchemy import inspect

In [174]:
def read_db_creds(creds_filepath):
    with open(creds_filepath) as f:
        creds = yaml.load(f, Loader=SafeLoader)
        return creds
    
read_db_creds('db_creds.yaml')

{'RDS_HOST': 'data-handling-project-readonly.cq2e8zno855e.eu-west-1.rds.amazonaws.com',
 'RDS_PASSWORD': 'AiCore2022',
 'RDS_USER': 'aicore_admin',
 'RDS_DATABASE': 'postgres',
 'RDS_PORT': 5432}

In [175]:
creds = read_db_creds('db_creds.yaml')
def init_db_engine(creds):
    engine = create_engine(f"postgresql://{creds['RDS_USER']}:{creds['RDS_PASSWORD']}@{creds['RDS_HOST']}:{creds['RDS_PORT']}/{creds['RDS_DATABASE']}")
    return engine.connect()

In [176]:
run_engine = init_db_engine(creds)
def list_db_tables():
        inspector = inspect(run_engine)
        table_names = inspector.get_table_names()
        return table_names

list_db_tables()

Exception during reset or similar
Traceback (most recent call last):
  File "/home/van28/anaconda3/envs/my_env/lib/python3.10/site-packages/sqlalchemy/pool/base.py", line 763, in _finalize_fairy
    fairy._reset(pool, transaction_was_reset)
  File "/home/van28/anaconda3/envs/my_env/lib/python3.10/site-packages/sqlalchemy/pool/base.py", line 1038, in _reset
    pool._dialect.do_rollback(self)
  File "/home/van28/anaconda3/envs/my_env/lib/python3.10/site-packages/sqlalchemy/engine/default.py", line 683, in do_rollback
    dbapi_connection.rollback()
psycopg2.OperationalError: SSL SYSCALL error: EOF detected



['legacy_store_details', 'legacy_users', 'orders_table']

In [177]:
def read_rds_tables(table_names):
    df = pd.read_sql_table(table_names[1],run_engine)
    return df

In [178]:
table_names = list_db_tables()
table = read_rds_tables(table_names)
print(table.head(5))

   index first_name last_name date_of_birth                    company  \
0      0   Sigfried     Noack    1990-09-30         Heydrich Junitz KG   
1      1        Guy     Allen    1940-12-01                    Fox Ltd   
2      2      Harry  Lawrence    1995-08-02  Johnson, Jones and Harris   
3      3     Darren   Hussain    1972-09-23                Wheeler LLC   
4      4      Garry     Stone    1952-12-20                 Warner Inc   

                  email_address  \
0             rudi79@winkler.de   
1  rhodesclifford@henderson.com   
2  glen98@bryant-marshall.co.uk   
3    daniellebryan@thompson.org   
4       billy14@long-warren.com   

                                             address         country  \
0                       Zimmerstr. 1/0\n59015 Gießen         Germany   
1  Studio 22a\nLynne terrace\nMcCarthymouth\nTF0 9GH  United Kingdom   
2                 92 Ann drive\nJoanborough\nSK0 6LR  United Kingdom   
3             19 Robinson meadow\nNew Tracy\nW22 2QG  Un

In [39]:
string_table = table.to_string('users.txt')

In [7]:
table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15320 entries, 0 to 15319
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          15320 non-null  int64 
 1   first_name     15320 non-null  object
 2   last_name      15320 non-null  object
 3   date_of_birth  15320 non-null  object
 4   company        15320 non-null  object
 5   email_address  15320 non-null  object
 6   address        15320 non-null  object
 7   country        15320 non-null  object
 8   country_code   15320 non-null  object
 9   phone_number   15320 non-null  object
 10  join_date      15320 non-null  object
 11  user_uuid      15320 non-null  object
dtypes: int64(1), object(11)
memory usage: 1.4+ MB


In [44]:
table.dtypes

index             int64
first_name       object
last_name        object
date_of_birth    object
company          object
email_address    object
address          object
country          object
country_code     object
phone_number     object
join_date        object
user_uuid        object
dtype: object

In [28]:
print(table.columns)
print(len(table.columns))

Index(['index', 'first_name', 'last_name', 'date_of_birth', 'company',
       'email_address', 'address', 'country', 'country_code', 'phone_number',
       'join_date', 'user_uuid'],
      dtype='object')
12


In [179]:
# How many NULL values?
count_null = 0
for index, row in table.iterrows():
    if row['first_name'] == 'NULL':
        count_null +=1
print(count_null)

21


In [44]:
null_values = table[table['first_name'].isnull()]
print(null_values)

Empty DataFrame
Columns: [index, first_name, last_name, date_of_birth, company, email_address, address, country, country_code, phone_number, join_date, user_uuid]
Index: []


In [180]:
# Solution: Eliminate NULL values

table_null = table[table.first_name != 'NULL']
print(len(table_null))
print(len(table))

15299
15320


In [181]:
# Deal with date formats
users_dates = table[['date_of_birth','join_date']]

In [77]:
users_dates.to_string('user_dates.txt')

In [182]:
# Solution: 
# Results in cleaned_dates.txt
import datetime
format = '%Y-%m-%d'

cleaned_dates = users_dates.apply(pd.to_datetime, infer_datetime_format=True, errors='coerce')
cleaned_dates.to_string('cleaned_dates.txt')

In [183]:
date_cols = table[['date_of_birth','join_date']]
users_df_clean = date_cols.apply(pd.to_datetime, 
                                          infer_datetime_format=True, # Fix dates
                                          errors='coerce')
users_df_clean.to_string('cleaned_dates.txt')

ValueError: Boolean array expected for the condition, not object

In [94]:
# Remove /n 
address_col = table['address']
address_col.to_string('address.txt')

In [132]:
address_col = table['address']
print(address_col.iloc[0])
print(address_col.iloc[0].replace('\n',' '))

Zimmerstr. 1/0
59015 Gießen
Zimmerstr. 1/0 59015 Gießen


In [151]:
# Solution: remove spaces \n
address_col = address_col.apply(lambda x:x.replace('\n', ','))
address_col.to_string('address_clean.txt')

In [166]:
address_col.to_csv('address_clean.csv')

In [172]:
check_address = pd.read_csv('address_clean.csv')
print(check_address['address'].iloc[113])

Studio 89W,Laura junctions,West Nicolashire,B0 5FU


In [29]:
actors = pd.read_sql_query('''SELECT * FROM actor LIMIT 10''', engine).set_index('actor_id')
actors

,first_name,last_name,last_update
actor_id,,,
1,PENELOPE,GUINESS,2006-02-15 09:34:33
2,NICK,WAHLBERG,2006-02-15 09:34:33
3,ED,CHASE,2006-02-15 09:34:33
4,JENNIFER,DAVIS,2006-02-15 09:34:33
5,JOHNNY,LOLLOBRIGIDA,2006-02-15 09:34:33
6,BETTE,NICHOLSON,2006-02-15 09:34:33
7,GRACE,MOSTEL,2006-02-15 09:34:33
8,MATTHEW,JOHANSSON,2006-02-15 09:34:33
9,JOE,SWANK,2006-02-15 09:34:33


Details: 

Create a string with the next value "Hello, my name is {name} and I am {age} years old" and store it in a variable called message.
For example, if the user enters John and 25, the output should be:
Hello, my name is John and I am 25 years old
Print the value of message using the print() function.

TODO - Create a variable called message that contains the mentioned string

name = input("Enter your name: ")
age = input("Enter your Age: ")
message = "Hello, my name is {name} and I am {age} years old"
print(f'message {name},{age}')

The output I am getting is :
message John,25.

Attempted Solution: 

I have tried it, but not getting the required solution. Please help

In [70]:
name = input("Enter your name: ")
age = input("Enter your Age: ")
message = "Hello, my name is {name} and I am {age} years old"
print(f'message {name},{age}')


message van,24


In [71]:
message = f"Hello, my name is {name} and I am {age} years old"
print(message)

Hello, my name is van and I am 24 years old
